[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/deJong_Mackay_et_al_2023/convergence.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=examples/PySDM_examples/deJong_Mackay_et_al_2023/convergence.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/deJong_Mackay_et_al_2023/convergence.ipynb)

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [2]:
import os
os.environ['NUMBA_NUM_THREADS'] = '10'

from open_atmos_jupyter_utils import show_plot
from datetime import datetime
import json
import numpy as np
from matplotlib import pyplot
import numba

from PySDM.backends import GPU, CPU
from PySDM.physics import si

from perf_comp.utils import go_benchmark, process_results, plot_processed_results, write_to_file
from perf_comp.utils import measure_time_per_timestep, measure_time_for_each_step, plot_time_per_step
from perf_comp.setups import setup_coalescence_only_sim, setup_breakup_only_sim, setup_coalescence_breakup_sim

In [3]:
TIMESTAMP = str(datetime.now()).replace(' ', '_')

In [4]:
SIM_RUN_FILENAME="env_name_" + TIMESTAMP

In [5]:
assert not os.path.isfile(SIM_RUN_FILENAME)

!echo NUMBA_DEFAULT_NUM_THREADS: $numba.config.NUMBA_DEFAULT_NUM_THREADS >> $SIM_RUN_FILENAME
!echo NUMBA_NUM_THREADS: $numba.config.NUMBA_NUM_THREADS >> $SIM_RUN_FILENAME
!lscpu >> $SIM_RUN_FILENAME
!nvidia-smi >> $SIM_RUN_FILENAME
!nvidia-smi -L >> $SIM_RUN_FILENAME

!cat /proc/cpuinfo >> $SIM_RUN_FILENAME

zsh:1: command not found: lscpu
zsh:1: command not found: nvidia-smi
zsh:1: command not found: nvidia-smi
cat: /proc/cpuinfo: No such file or directory


In [6]:
SIM_RUN_FILENAME

'env_name_2023-09-29_01:35:17.451690'

In [7]:
CI = True #'CI' in os.environ

exponents = [3, 5, 8, 10, 12, 14, 16, 18, 20, 22, 24] if not CI else [3, 5]
n_sds = [2 ** i for i in exponents]

numba_n_threads = [1, 2, 4, 5, 6, 8, 10] if not CI else [1, 2]

n_realisations = 3 if not CI else 2
seeds = list(range(n_realisations))

n_steps_short = 100 if not CI else 10
n_steps_full = 2048 if not CI else 20

# Benchmark regular setup (without scaling)

### Coalescence-only

In [8]:
res_coalescence_only = go_benchmark(
    setup_coalescence_only_sim, n_sds, n_steps_short, seeds, numba_n_threads=numba_n_threads, double_precision=True, 
    sim_run_filename=SIM_RUN_FILENAME + '-coalescence',
    backends=[CPU, GPU]
)


 before

Numba_1 8


== Settings ==
n_sd 8
total_number 1000000000000.0
dv 1.0


prod before
total_number [1.e+12]
total_volume [1000000.]
super_particle_count [8.]

start

 after
total_number [4.88281251e+08]
total_volume [1000000.]
super_particle_count [8.]

== Settings ==
n_sd 8
total_number 1000000000000.0
dv 1.0


prod before
total_number [1.e+12]
total_volume [1000000.]
super_particle_count [8.]

start

 after
total_number [4.88281251e+08]
total_volume [1000000.]
super_particle_count [8.]

Numba_1 32


== Settings ==
n_sd 32
total_number 1000000000000.0
dv 1.0


prod before
total_number [1.e+12]
total_volume [1000000.]
super_particle_count [32.]

start

 after
total_number [7.32412423e+08]
total_volume [1000000.]
super_particle_count [32.]

== Settings ==
n_sd 32
total_number 1000000000000.0
dv 1.0


prod before
total_number [1.e+12]
total_volume [1000000.]
super_particle_count [32.]

start

 after
total_number [8.23961372e+08]
total_volume [1000000.]
super_particle_count [32.]


/Users/aleks/uni/PySDM/PySDM/backends/thrust_rtc.py:72: UserWarning: CUDA is not available, using FakeThrustRTC!
  warnings.warn("CUDA is not available, using FakeThrustRTC!")


total_number [1.e+12]
total_volume [1000000.]
super_particle_count [8.]

start

 after
total_number [4.88281251e+08]
total_volume [1000000.]
super_particle_count [8.]

== Settings ==
n_sd 8
total_number 1000000000000.0
dv 1.0


prod before


/Users/aleks/uni/PySDM/PySDM/backends/thrust_rtc.py:72: UserWarning: CUDA is not available, using FakeThrustRTC!
  warnings.warn("CUDA is not available, using FakeThrustRTC!")


total_number [1.e+12]
total_volume [1000000.]
super_particle_count [8.]

start

 after
total_number [4.88281251e+08]
total_volume [1000000.]
super_particle_count [8.]

ThrustRTC 32


== Settings ==
n_sd 32
total_number 1000000000000.0
dv 1.0


prod before


/Users/aleks/uni/PySDM/PySDM/backends/thrust_rtc.py:72: UserWarning: CUDA is not available, using FakeThrustRTC!
  warnings.warn("CUDA is not available, using FakeThrustRTC!")


total_number [1.e+12]
total_volume [1000000.]
super_particle_count [32.]

start


KeyboardInterrupt: 

In [ ]:
coalescence_only_processed = process_results(res_coalescence_only)

In [ ]:
plot_processed_results(coalescence_only_processed, plot_title=f'coalescence-only (n_steps: {n_steps_short})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-coalescence-double-n_steps{n_steps_short}.txt"
write_to_file(filename=filename, d=coalescence_only_processed)

### Breakup-only

In [ ]:
res_breakup_only = go_benchmark(
    setup_breakup_only_sim, n_sds, n_steps_short, seeds, numba_n_threads=numba_n_threads, double_precision=True,
    sim_run_filename=SIM_RUN_FILENAME + '-breakup',
    backends=[CPU, GPU]
)

In [ ]:
breakup_only_processed = process_results(res_breakup_only)

In [ ]:
plot_processed_results(breakup_only_processed, plot_title=f'breakup-only  (n_steps: {n_steps_short})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-breakup-double-n_steps{n_steps_short}.txt"
write_to_file(filename=filename, d=breakup_only_processed)

### Coalescence and Breakup

In [ ]:
res_coal_breakup = go_benchmark(
    setup_coalescence_breakup_sim, n_sds, n_steps_full, seeds, numba_n_threads=numba_n_threads, double_precision=True, 
    sim_run_filename=SIM_RUN_FILENAME + '-coal-break',
    backends=[CPU, GPU]
)

In [ ]:
coal_breakup_processed = process_results(res_coal_breakup)

In [ ]:
plot_processed_results(coal_breakup_processed, plot_title=f'coalescence+breakup (n_steps: {n_steps_full})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-coal_with_breakup-double-n_steps{n_steps_full}.txt"
write_to_file(filename=filename, d=coal_breakup_processed)

# Benchmark setup with scaling

In [ ]:
def total_number_from_n_sd(n_sd):
    return n_sd * 1e8

def dv_from_n_sd(n_sd):
    return n_sd * (0.125 * si.m**3)

### Coalescence-only

In [ ]:
res_coalescence_only_scaled = go_benchmark(
    setup_coalescence_only_sim, n_sds, n_steps_short, seeds, numba_n_threads=numba_n_threads, double_precision=True, 
    sim_run_filename=SIM_RUN_FILENAME + '-coalescence-scaled',
    total_number=total_number_from_n_sd,
    dv=dv_from_n_sd,
    backends=[CPU, GPU]
)

In [ ]:
coalescence_only_processed_scaled = process_results(res_coalescence_only_scaled)

In [ ]:
plot_processed_results(coalescence_only_processed_scaled, plot_title=f'coalescence-only with scaling (n_steps: {n_steps_short})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-scaled-coalescence-double-n_steps{n_steps_short}.txt"
write_to_file(filename=filename, d=coalescence_only_processed_scaled)

### Breakup-only

In [ ]:
res_breakup_only_scaled = go_benchmark(
    setup_breakup_only_sim, n_sds, n_steps_short, seeds, numba_n_threads=numba_n_threads, double_precision=True,
    sim_run_filename=MY_SETUP_INFO_FILENAME + '-breakup-scaled',
    total_number=total_number_from_n_sd,
    dv=dv_from_n_sd,
    backends=[CPU, GPU]
)

In [ ]:
breakup_only_processed_scaled = process_results(res_breakup_only_scaled)

In [ ]:
plot_processed_results(breakup_only_processed_scaled, plot_title=f'breakup-only with scaling (n_steps: {n_steps_short})')

In [ ]:
filename=f"{MY_SETUP_INFO_FILENAME}-results-scaled-breakup-double-n_steps{n_steps_short}.txt"
write_to_file(filename=filename, d=breakup_only_processed_scaled)

### Coalescence and Breakup

In [ ]:
res_coal_breakup_scaled = go_benchmark(
    setup_coalescence_breakup_sim, n_sds, n_steps_full, seeds, numba_n_threads=numba_n_threads, double_precision=True, 
    sim_run_filename=SIM_RUN_FILENAME + '-coal-break-scaled',
    total_number=total_number_from_n_sd,
    dv=dv_from_n_sd,
    backends=[CPU, GPU]
)

In [ ]:
coal_breakup_processed_scaled = process_results(res_coal_breakup_scaled)

In [ ]:
plot_processed_results(coal_breakup_processed_scaled, plot_title=f'coalescence+breakup with scaling (n_steps: {n_steps_full})')

In [ ]:
filename=f"{SIM_RUN_FILENAME}-results-scaled-coal_with_breakup-double-n_steps{n_steps_full}.txt"
write_to_file(filename=filename, d=coal_breakup_processed_scaled)